In [ ]:
import numpy as np

### ID classification

In this method, TDs and RDs satisfy $ \frac{ |B_N| }{ |B_{bg}| } < 0.2$ and $ | \frac{ \Delta |B| }{ |B_{bg}| } | > 0.4$ B BN bg ∣∣ ∣∣ , < D 0.2 B B bg ∣∣ ∣ ∣ , respectively. Moreover, IDs with < 0.4 B BN bg ∣∣ ∣∣ , < D 0.2 B B bg ∣∣ ∣ ∣ could be either TDs or RDs, and so are termed EDs. Similarly, NDs are defined as > 0.4 B BN bg ∣∣ ∣∣ , > D 0.2 B B bg ∣∣ ∣ ∣ because they can be neither TDs nor RDs. It is worth noting that EDs and NDs here are not physical concepts like RDs and TDs. RDs or TDs correspond to specific types of structures in the MHD framework, while EDs and NDs are introduced just to better quantify the statistical results.


Criteria Used to Classify Discontinuities on the Basis of Magnetic Data Type

| Type   |  $\|B_n/B\|$      | $\| \Delta B / B \|$  |
|----------|-------------|------|
| Rotational (RD) | large | small |
| Tangential (TD) | small |  large |
| Either (ED) | small | small |
| Neither (ND) | large | large |


In [ ]:
BnOverB_RD_lower_threshold = 0.4
dBOverB_RD_upper_threshold = 0.2

BnOverB_TD_upper_threshold = 0.2
dBOverB_TD_lower_threshold = dBOverB_RD_upper_threshold

BnOverB_ED_upper_threshold = BnOverB_RD_lower_threshold
dBOverB_ED_upper_threshold = dBOverB_TD_lower_threshold

BnOverB_ND_lower_threshold = BnOverB_TD_upper_threshold
dBOverB_ND_lower_threshold = dBOverB_RD_upper_threshold

In [ ]:
def classify_id(BnOverB, dBOverB):
    BnOverB = np.abs(np.asarray(BnOverB))
    dBOverB = np.asarray(dBOverB)

    s1 = (BnOverB > BnOverB_RD_lower_threshold)
    s2 = (dBOverB > dBOverB_RD_upper_threshold)
    s3 = (BnOverB > BnOverB_TD_upper_threshold)
    s4 = s2 # note: s4 = (dBOverB > dBOverB_TD_lower_threshold)
    
    RD = s1 & ~s2
    TD = ~s3 & s4
    ED = ~s1 & ~s4
    ND = s3 & s2

    # Create an empty result array with the same shape
    result = np.empty_like(BnOverB, dtype=object)

    result[RD] = "RD"
    result[TD] = "TD"
    result[ED] = "ED"
    result[ND] = "ND"

    return result

## Classification of candidates

In [ ]:
sns.jointplot(
    candidates_jno_tau_60s,
    x='dBOverB', y='BnOverB',
    # kind='kde',
    kind="hex",
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 sns.jointplot(                                                                               │
│ ❱ 2 │   candidates_jno_tau_60s,                                                                  │
│   3 │   x='dBOverB', y='BnOverB',                                                                │
│   4 │   # kind='kde',                                                                            │
│   5 │   kind="hex",                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'candidates_jno_tau_60s' is not defined

In [ ]:
sns.jointplot(
    all_candidates,
    x='dBOverB', y='BnOverB',
    # kind='kde',
    kind="hex",
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 sns.jointplot(                                                                               │
│ ❱ 2 │   all_candidates,                                                                          │
│   3 │   x='dBOverB', y='BnOverB',                                                                │
│   4 │   # kind='kde',                                                                            │
│   5 │   kind="hex",                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'all_candidates' is not defined

### Distribution of the DD types

##### Plot distribution of types for each missions


In [ ]:
alt.Chart(all_candidates).encode(
    x=alt.X("count()").stack("normalize").title("Share of ID types"),
    y=alt.Y('sat').title(None),
    color='type',
).mark_bar()

# alt.Chart(distributions).encode(
#     alt.X('ratio:Q', title='DD type distribution').axis(format='.0%'),
#     y=alt.Y('sat', title=None),
#     color='type',
# ).mark_bar()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 alt.Chart(all_candidates).encode(                                                           │
│    2 │   x=alt.X("count()").stack("normalize").title("Share of ID types"),                       │
│    3 │   y=alt.Y('sat').title(None),                                                             │
│    4 │   color='type',                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'alt' is not defined

In [ ]:
distributions = all_candidates.group_by("sat", "type").agg(pl.count()).with_columns(
    (pl.col("count") / pl.sum("count").over("sat")).alias("ratio")
)
count_table = distributions.to_pandas().pivot(index='sat', columns='type', values='count')[['RD', 'TD', 'ED', 'ND']]
ratio_table = distributions.to_pandas().pivot(index='sat', columns='type', values='ratio')[['RD', 'TD', 'ED', 'ND']]
# display(distributions.to_pandas().pivot(index='sat', columns='type', values='ratio')[['RD', 'TD', 'ED', 'ND']].style.format("{:.0%}"))
display(count_table, ratio_table.style.format("{:.0%}"))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 distributions = all_candidates.group_by("sat", "type").agg(pl.count()).with_columns(         │
│   2 │   (pl.col("count") / pl.sum("count").over("sat")).alias("ratio")                           │
│   3 )                                                                                            │
│   4 count_table = distributions.to_pandas().pivot(index='sat', columns='type', values='count     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'all_candidates' is not defined

##### Plot distribution of types for each missions **over time**

In [ ]:
alt.Chart(all_candidates).mark_bar(binSpacing=0).encode(
    x="yearmonth(time)",
    y=alt.Y("count()").stack("normalize").title("Share of ID types"),
    row=alt.Row("sat").title(None),
    color="type",
).configure_axis(grid=False).properties(height=100)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 alt.Chart(all_candidates).mark_bar(binSpacing=0).encode(                                     │
│   2 │   x="yearmonth(time)",                                                                     │
│   3 │   y=alt.Y("count()").stack("normalize").title("Share of ID types"),                        │
│   4 │   row=alt.Row("sat").title(None),                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'alt' is not defined

In [ ]:
%%R
plot_type_distribution <- function(data, bin_width = 30) { 
    data$date_only <- as.Date(data$time)
    
    p <- ggplot(data, aes(date_only, fill = type)) +
        geom_histogram(binwidth = bin_width, position = "fill") + 
        theme_pubr(base_size = 16)
        
    return(p)
}

p1 <- plot_type_distribution(jno_candidates)
p2 <- plot_type_distribution(thb_candidates)
p3 <- plot_type_distribution(sta_candidates)

p <- ggarrange(
    p1 + rremove("xlab"),
    p2 + rremove("xlab"), p3, 
    nrow = 3, align = 'hv', 
    labels=list("JUNO", "ARTEMIS-B", "STEREO-A"), hjust=0, vjust=0,
    legend = 'right', common.legend = TRUE
)
# save_plot(filename = "type_distribution")
p

UsageError: Cell magic `%%R` not found.
